# <font color="red"> ACCESSING YT API | YT ID | RETRIEVE COMMENTS


In [1]:
%%capture
!pip install -r requirements.txt

In [5]:
# Filter out warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from googleapiclient.discovery import build
import os

API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise RuntimeError("Missing API KEY")
# youtube = build('youtube', 'v3', developerKey=API_KEY)

In [7]:
import numpy as np
import pandas as pd
import requests

In [ ]:
VIDEO_IDs = ['XqgMWVZvi7w','5530I_pYjbo','P6FORpg0KVo','iNyUmbmQQZg','TjPFZaMe2yw']

video_dict = {}

# def fetch_comments_from_videos(video_dict, max_results=100):
#     video_dict = {}

#     for video_id in VIDEO_IDs:
#         params = {
#             'part': 'snippet',
#             'videoId': video_id,
#             'maxResults': min(max_results,100),  # Max per page (use pagination to get more)
#             'textFormat': 'plainText',
#             'key': API_KEY
#         }

#         response = requests.get('https://www.googleapis.com/youtube/v3/commentThreads', params=params, timeout=20)
#         data = response.json()

#         # Add error handling to check if the request was successful and if 'items' exists
#         if response.status_code == 200 and 'items' in data:
#             def get_comments(data):
#                 comments = []
#                 for d in data['items']:
#                     comments.append(d['snippet']['topLevelComment']['snippet']['textDisplay'])
#                 return comments

#             comments = get_comments(data)
#             video_dict[video_id] = comments
#         else:
#             print(f"Error for video {video_id}: {data.get('error', {}).get('message', 'Unknown error')}")
#             pass

# fetch_comments_from_videos(video_dict, max_results=100)

# # Remove videos where the number of comments is less than max_results
# keys_to_remove = [k for k, v in video_dict.items() if len(v) != max_results]
# for k in keys_to_remove:
#     video_dict.pop(k)



def fetch_comments_from_videos(video_ids, max_results=100):
    video_dict = {}

    for video_id in video_ids:
        params = {
            "part": "snippet",
            "videoId": video_id,
            "maxResults": min(max_results, 100),
            "textFormat": "plainText",
            "key": API_KEY,
        }

        resp = requests.get(
            "https://www.googleapis.com/youtube/v3/commentThreads",
            params=params,
            timeout=20
        )
        data = resp.json()

        if resp.status_code != 200:
            msg = data.get("error", {}).get("message", "Unknown error")
            print(f"Error for video {video_id}: {msg}")
            video_dict[video_id] = []
            continue

        comments = [
            item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
            for item in data.get("items", [])
        ]
        video_dict[video_id] = comments

    return video_dict

video_dict = fetch_comments_from_videos(VIDEO_IDs, max_results=100)

# Drop empty ones
video_dict = {k: v for k, v in video_dict.items() if len(v) > 0}

In [ ]:
record = []

for video_id, comments in video_dict.items():
    for indx, comment in enumerate(comments):
        record.append({
            "video_id": video_id,
            "index": indx,
            "comment": comment,
            "Sentiment score": None
        })

df = pd.DataFrame(record)

,video_id,index,comment,Sentiment score
0,XqgMWVZvi7w,0,"Rev counter cannot catch up gearbox work 😄, i ...",None
1,XqgMWVZvi7w,1,I don't understand. I don't think the GT is a ...,None
2,XqgMWVZvi7w,2,"The camera angle is unfortunately very, very b...",None
3,XqgMWVZvi7w,3,My God what a car,None
4,XqgMWVZvi7w,4,Dream ❤,None
...,...,...,...,...
395,TjPFZaMe2yw,95,We'll discuss the secret to that group sucess,None
396,TjPFZaMe2yw,96,Also stay sober. Don't go on benders. It disru...,None
397,TjPFZaMe2yw,97,Medhya churna of Planet Ayurveda can help in e...,None
398,TjPFZaMe2yw,98,hehe I just gave up hehehehehehehehehehehehehehe,None


# <font color="red"> PRE-PROCESSING DATA


### <font color="aqua"> 1. CONVERTING TO LOWERCASE

In [ ]:
## convert to lowercase
df['comment'] = df['comment'].str.lower()
df

,video_id,index,comment,Sentiment score
0,XqgMWVZvi7w,0,"rev counter cannot catch up gearbox work 😄, i ...",None
1,XqgMWVZvi7w,1,i don't understand. i don't think the gt is a ...,None
2,XqgMWVZvi7w,2,"the camera angle is unfortunately very, very b...",None
3,XqgMWVZvi7w,3,my god what a car,None
4,XqgMWVZvi7w,4,dream ❤,None
...,...,...,...,...
395,TjPFZaMe2yw,95,we'll discuss the secret to that group sucess,None
396,TjPFZaMe2yw,96,also stay sober. don't go on benders. it disru...,None
397,TjPFZaMe2yw,97,medhya churna of planet ayurveda can help in e...,None
398,TjPFZaMe2yw,98,hehe i just gave up hehehehehehehehehehehehehehe,None


### <font color="aqua"> 2. NEGATION HANDLING

In [ ]:
##converting into lexicons
import contractions

# Expand contractions for each comment before tokenizing
df['comment']= df['comment'].map(lambda x: contractions.fix(x) if isinstance(x, str) else x)
# df

### <font color="aqua"> 3. REMOVING UNNECESSARY CHARACTERS | ISOLATING EMOJIS | FILTERING OUT NON-ENGLISH COMMENTS

In [ ]:
import re
def clean_comments(df):
    df['comment'] = df['comment'].map(lambda x: re.sub(r'\d+', '', x) if isinstance(x, str) else x) # Remove numbers
    df['comment'] = df['comment'].map(lambda x: re.sub(r'https?://\S+', '', x) if isinstance(x, str) else x) # Remove links
    df['comment'] = df['comment'].map(lambda x: re.sub(r'[.,!?\"@#&$%()\-:;/\[\]{{}}<]', '', x) if isinstance(x, str) else x) # Remove links
    return df

df = clean_comments(df)

In [ ]:
## ADDING SPACE AROUND EMOJIS TO TOKENIZE BETTER
def add_space_around_emoji(text):
    pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags
        "\U00002700-\U000027BF"  # Dingbats
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U00002600-\U000026FF"  # Misc symbols
        "]", flags=re.UNICODE)
    return pattern.sub(lambda x: f" {x.group(0)} ", text)

df['comment'] = df['comment'].map(lambda x: add_space_around_emoji(x) if isinstance(x, str) else x)
# df


In [ ]:
# FILTER OUT NON-ENGLISH COMMENTS
def filter_non_english_words_keep_emojis(text):
    # Emoji unicode range
    emoji_ranges = (
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        "\U0001F680-\U0001F6FF"  # Transport & Map
        "\U0001F1E0-\U0001F1FF"  # Flags
        "\U00002700-\U000027BF"  # Dingbats
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols
        "\U00002600-\U000026FF"  # Misc symbols
    )

    # Pattern to keep English words and emojis
    emoji_pattern = f"[{emoji_ranges}]"
    english_word_pattern = r"\b[a-zA-Z]+\b"

    # Find all English words and emojis
    english_words = re.findall(english_word_pattern, text)
    emojis = re.findall(emoji_pattern, text)

    return ' '.join(english_words + emojis)

df['comment'] = df['comment'].map(lambda x: filter_non_english_words_keep_emojis(x) if isinstance(x, str) else x)
# df

### <font color="aqua"> 4. TOKENIZE COMMENTS

In [ ]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
df["comment"] = df["comment"].apply(lambda x: word_tokenize(x) if isinstance(x, str) else x)

### <font color="aqua"> 5. REMOVE STOPWORDS

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
tmp = df.copy()
negations = {
    "not", "nor", "no",
    "do", "are", "cannot", "could", "did", "does",
    "had", "has", "have", "is", "might", "must",
    "need", "should", "was", "were", "would", "can"
}
stop_words = set(stopwords.words('english'))

stop_words = [word for word in stop_words if word not in negations]

# df["comment"] = df["comment"].map(lambda x: [word for word in x if word not in stop_words] if isinstance(x, list) else x)
tmp = tmp.map(lambda x: [word for word in x if word not in stop_words] if isinstance(x, list) else x)

df = tmp.copy()

# <font color="red">Build/Train the Model
## Use BERT and HuggingFace or VADAR


### <font color="cyan"> 1. LOADING PRE-TRAINED MODEL FROM HUGGINGFACE

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = TFAutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
df = df.map(lambda x: " ".join(x) if isinstance(x, list) else x)
df

,video_id,index,comment,Sentiment score
0,XqgMWVZvi7w,0,rev counter can not catch gearbox work think i...,None
1,XqgMWVZvi7w,1,do not understand do not think gt is rwd,None
2,XqgMWVZvi7w,2,camera angle is unfortunately bad although und...,None
3,XqgMWVZvi7w,3,god car,None
4,XqgMWVZvi7w,4,dream ❤,None
...,...,...,...,...
395,TjPFZaMe2yw,95,discuss secret group sucess,None
396,TjPFZaMe2yw,96,also stay sober do not go benders disrupts bra...,None
397,TjPFZaMe2yw,97,medhya churna planet ayurveda can help enhanci...,None
398,TjPFZaMe2yw,98,hehe gave hehehehehehehehehehehehehehe,None


In [ ]:
tokenized = tokenizer(
    df["comment"].tolist(),
    return_tensors="tf",
    padding=True,
    truncation=True,
    max_length=128
)
tokenized

{'input_ids': <tf.Tensor: shape=(400, 128), dtype=int32, numpy=
array([[    0, 23559,  3231, ...,     1,     1,     1],
       [    0,  5016,    45, ...,     1,     1,     1],
       [    0, 25092, 11792, ...,     1,     1,     1],
       ...,
       [    0,  4567, 18321, ...,     1,     1,     1],
       [    0,   700,   700, ...,     1,     1,     1],
       [    0, 12805,   225, ...,     1,     1,     1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(400, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [ ]:
# # Get model outputs
# outputs = model(tokenized)
# outputs

dataset = tf.data.Dataset.from_tensor_slices(tokenized).batch(32) # Choose a batch size that fits in memory

predictions = []
confidences = []

# Iterate over the dataset in batches and get predictions
for batch in dataset:
    outputs = model(batch)
    prediction = tf.nn.softmax(outputs.logits, axis=1)
    predictions.append(prediction.numpy())

# Concatenate the predictions from all batches
predictions = np.concatenate(predictions, axis=0)

# Get the predicted classes and confidence scores
pred_classes = tf.argmax(predictions, axis=1).numpy()
labels = ['negative', 'neutral', 'positive']
pred_labels = [labels[i] for i in pred_classes]
confidences = predictions.max(axis=1)

# %%
results_df = pd.DataFrame({
    "comment": df["comment"],  # Convert first row of df to list
    "sentiment": pred_labels,
    "confidence": confidences
})
results_df

,comment,sentiment,confidence
0,rev counter can not catch gearbox work think i...,negative,0.377174
1,do not understand do not think gt is rwd,negative,0.384964
2,camera angle is unfortunately bad although und...,negative,0.365956
3,god car,negative,0.388468
4,dream ❤,negative,0.406690
...,...,...,...
395,discuss secret group sucess,positive,0.385340
396,also stay sober do not go benders disrupts bra...,negative,0.388172
397,medhya churna planet ayurveda can help enhanci...,negative,0.389105
398,hehe gave hehehehehehehehehehehehehehe,positive,0.389801


In [ ]:
model.save_pretrained("roberta_web_model")
tokenizer.save_pretrained("roberta_web_model")

('roberta_web_model/tokenizer_config.json',
 'roberta_web_model/special_tokens_map.json',
 'roberta_web_model/vocab.json',
 'roberta_web_model/merges.txt',
 'roberta_web_model/added_tokens.json',
 'roberta_web_model/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# prompt: how to save the model to google drive

import os

# Define the path to save the model in your Google Drive
model_save_path = '/content/drive/MyDrive/my_sentiment_model'

# Create the directory if it doesn't exist
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(model_save_path)

# Save the TensorFlow model
model.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")


Model and tokenizer saved to /content/drive/MyDrive/my_sentiment_model


In [ ]:
import shutil
# shutil.make_archive('/content/drive/MyDrive/my_sentiment_model', 'zip', '/content/roberta_web_model')
shutil.make_archive("roberta_web_model", 'zip', "roberta_web_model")


'/content/roberta_web_model.zip'

In [ ]:
import streamlit as st

st.title("YouTube Sentiment Analyzer")
url = st.text_input("Enter YouTube URL")

if st.button("Fetch Comments"):
    video_id = get_video_details(url)
    if video_id:
        with st.spinner("Fetching comments..."):
            comments = get_video_details(video_id)
            st.success(f"Fetched {len(comments)} comments.")
            for c in comments:
                st.write(f"- {c}")
    else:
        st.error("Invalid YouTube URL")
